In [ ]:
# !pip install selenium
# !pip install webdriver_manager
# !pip install pymysql

In [2]:
# !pip install webdriver_manager
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

import time
from bs4 import BeautifulSoup
import re
import pandas as pd
import numpy as np
from tqdm import tqdm
import os
import pymysql
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [2]:
def page_down():
    global error
    global end_crawling
    while True:
        print(100)
        driver.switch_to.window(driver.window_handles[0])
        time.sleep(1)
        print(200)
        current_scroll = driver.execute_script('return window.pageYOffset')
        driver.execute_script(f'window.scrollTo(0, {current_scroll - 50});')
        time.sleep(1)
        print(300)
        driver.execute_script(f'window.scrollTo(0, {current_scroll + 150});')
        time.sleep(1)
        down_scroll = driver.execute_script('return window.pageYOffset')
        time.sleep(1)
        print(400)
        if current_scroll == down_scroll:
            print(500)
            driver.execute_script(f'window.scrollTo(0, {current_scroll - 50});')
            print(600)
            current_scroll = driver.execute_script('return window.pageYOffset')
            if current_scroll == down_scroll:
                print(600)
                error = True
                break
            else:
                print(700)
                end_crawling = True
                break
        try:
            print(800)
            driver.find_element(By.XPATH, value=f'//*[@id="__next"]/div[3]/div[2]/ul/li[{page_num}]/div/a/div[1]').click()
            driver.switch_to.window(driver.window_handles[0])
            craw()
            print(900)
            driver.back()
            time.sleep(1)
            driver.switch_to.window(driver.window_handles[0])
            break
        except:
            print(1000)
            return page_down()
    return

In [3]:
def page_down2():
    global error
    global end_crawling
    while True:
        try:
            print(2100)
            current_scroll2 = driver.execute_script('return window.pageYOffset')
            driver.execute_script(f'window.scrollTo(0, {current_scroll2 - 50});')
            time.sleep(1)
            print(2200)
            driver.execute_script(f'window.scrollTo(0, {current_scroll2 + 150});')
            time.sleep(1)
            down_scroll2 = driver.execute_script('return window.pageYOffset')
            time.sleep(1)
            print(2300)
            if current_scroll2 == down_scroll2:
                print(2400)
                error = True
                break
            try:
                print(2500)
                driver.find_element(By.XPATH, value='//*[@id="__next"]/main/div[1]/div/section/section/article[1]/div/button').click()
                break
            except:
                print(2600)
                return page_down2()
        except:
            return
    return

In [4]:
def craw():
    for j in range(1,6):
        time.sleep(3)
        if j == 1:
            print(1)
            try:
                element = driver.find_element(By.CLASS_NAME, value = 'JobHeader_JobHeader__DBrFJ')

                name.append(element.text.split('\n')[0].split('∙')[0])
                loc.append(element.text.split('\n')[0].split('∙')[1])
                career.append(element.text.split('\n')[0].split('∙')[2])
                title.append(element.text.split('\n')[1])
            except:
                name.append('')
                loc.append('')
                career.append('')
                title.append('')
        elif j == 2:
            print(2)
            try:
                element = driver.find_element(By.CLASS_NAME, value = 'CompanyTags_CompanyTags__wpSCo')
                if len(element.text.split('\n')) >=1:
                    tag.append(element.text.split('\n')[1:])
                else:
                    tag.append('')
            except:
                tag.append('')
        elif j == 3:
            print(3)
            try:
                element = driver.find_element(By.CLASS_NAME, value = 'JobDueTime_JobDueTime__iKbEO')
                if len(element.text.split('\n')) ==2:
                    timeout.append(element.text.split('\n')[1])
                else:
                    timeout.append('')
            except:
                timeout.append('')
        elif j ==4:
            print(4)
            try:
                element = driver.find_element(By.CLASS_NAME, value = 'JobWorkPlace_JobWorkPlace__5fs5x')
                if len(element.text.split('\n')) == 2:
                    loc2.append(element.text.split('\n')[1])
                else:
                    loc2.append('')
            except:
                loc2.append('')
        else:
            print(5)
            try:
                element = driver.find_element(By.CLASS_NAME, value='JobDescription_JobDescription__VWfcb')
                if '상세 정보 더 보기' in element.text:
                        page_down2()
                        element = driver.find_element(By.CLASS_NAME, value='JobDescription_JobDescription__VWfcb')
                        time.sleep(1)
                        detail.append(element.text)
            except:
                detail.append('')

In [5]:
def reset():
    global title
    global name
    global career
    global loc
    global tag
    global timeout
    global loc2
    global detail

    global done

    global current_scroll
    global down_scroll
    
    global current_scroll2
    global down_scroll2
    
    title = []           #채용공고
    name = []            #기업명
    career = []          #경력
    loc = []             #근무지
    tag = []             #태그
    timeout = []         #마감일
    loc2 = []            #근무지(상세)
    detail = []          #내용상세


    done = False

    # 스크롤 이동변수
    current_scroll = 0
    down_scroll = 0
    current_scroll2 = 0
    down_scroll2 = 0
    

In [6]:
def DF_make(file_name):
    tag_str = []
    df = pd.DataFrame()
    for i in tag:
         tag_str.append(','.join(i))

    df['기업명'] = name
    df['채용공고'] = title
    df['위치'] = loc
    df['위치상세'] = loc2
    df['태그'] = tag_str
    df['마감일'] = timeout
    df['경력'] = career
    df['채용상세'] = detail
    
    df = df.drop_duplicates()
    df = df.fillna('')
    df.to_csv(f'C:\\Users\\EZEN\pro\\pro_data\\pro_{file_name}.csv',encoding='utf-8')



In [7]:
def DB_insert(file_name):
    df = pd.read_csv(f'C:\\Users\\EZEN\pro\\pro_data\\pro_{file_name}.csv',encoding='utf-8')
    df = df.fillna('')
    conn = pymysql.connect(host='127.0.0.1', user='user', password='user1234', db='dbmaria', charset='utf8')
    cur = conn.cursor()
    cur.execute(f'''CREATE TABLE IF NOT EXISTS pro_{file_name}2 (
                   name VARCHAR(255),
                   title VARCHAR(255),
                   time VARCHAR(255),
                   loc VARCHAR(255),
                   career VARCHAR(255),
                   info VARCHAR(255),
                   loc2 VARCHAR(255),
                   detail TEXT(20000)
                )''')

    for index, row in df.iterrows():
        cur.execute(f"""
            INSERT INTO pro_{file_name}2 (
               name,
               title,
               time,
               loc,
               career,
               info,
               loc2,
               detail
            ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s)
        """, (
            row['기업명'],
            row['채용공고'],
            row['마감일'],
            row['위치'],
            row['경력'],
            row['태그'],
            row['위치상세'],
            row['채용상세']
        ))
    # 변경사항 커밋
    conn.commit()

    # 연결 종료
    conn.close()

In [8]:
url_list = ['https://www.wanted.co.kr/wdlist/518/1634?country=kr&job_sort=job.recommend_order&years=-1&selected=1634&locations=seoul.gangnam-gu&locations=seoul.guro-gu&locations=seoul.geumcheon-gu&locations=seoul.seocho-gu&locations=seoul.songpa-gu']

url_name_list = ['wanted_ML_engineer']
file_url_name = dict(zip(url_list,url_name_list))

In [9]:
end_num = 400

for page_url,file_name in file_url_name.items():
    print('십육')
    driver = webdriver.Chrome("")
    time.sleep(10)
    print('십칠')
    driver.get(page_url)
    time.sleep(10)
    print('십팔')
    error = False
    page_num2 = 1
    end_crawling = False
    reset()
    print('십구')
    while True:
        print('십팔')
        try:
            for page_num in tqdm(range(page_num2,end_num+1)):
                if(page_num % 20) == 0:
                    print('십삼')
                    time.sleep(5)
                    driver.switch_to.window(driver.window_handles[0])
                    time.sleep(5)
                    print('십사')
                    driver.quit()
                    time.sleep(5)
                    driver = webdriver.Chrome("")
                    time.sleep(5)
                    print('십오')
                    driver.get(page_url)
                    time.sleep(5)
                try:
                    print('a')
                    driver.find_element(By.XPATH, value=f'//*[@id="__next"]/div[3]/div[2]/ul/li[{page_num}]/div/a/div[1]').click()
                    driver.switch_to.window(driver.window_handles[0])
                    print('일')
                    craw()
                    print('이')
                    driver.back()
                    time.sleep(3)
                    driver.switch_to.window(driver.window_handles[0])
                except:
                    print('구')
                    page_down()
                if error == True:
                    print('십')
                    print(에러만들기)
                    print('십일')
                elif end_crawling == True:
                    print('십이')
                    break

        except:
            print('삼')
            time.sleep(5)
            driver.switch_to.window(driver.window_handles[0])
            time.sleep(5)
            print('사')
            driver.quit()
            time.sleep(5)
            driver = webdriver.Chrome("")
            print('오')
            time.sleep(5)
            driver.get(page_url)
            time.sleep(5)
            print('육')
            page_num2 = page_num

        if (end_crawling ==True) or (page_num ==end_num):
            print('칠')
            DF_make(file_name)
            DB_insert(file_name)
            page_num2 = 1
            print('팔')
            break

십육
십칠
십팔
십구
십팔


  0%|                                                                                          | 0/400 [00:00<?, ?it/s]

a
일
1
2
3
4
5
2100
2200
2300
2500
2600
2100
2200
2300
2500
이


  0%|▏                                                                               | 1/400 [00:27<3:00:28, 27.14s/it]

a
일
1
2
3
4
5
2100
2200
2300
2500
2600
2100
2200
2300
2500
이


...

 54%|████████████████████████████████████████▋                                   | 214/400 [2:26:04<1:27:33, 28.24s/it]

a
구
100
200
300
400
800
1000
100
200
300


 54%|████████████████████████████████████████▋                                   | 214/400 [2:26:13<2:07:05, 41.00s/it]

400
500
600
700
십이
칠


팔


In [10]:
df

NameError: name 'df' is not defined

In [76]:
print(len(name))
print(len(title))
print(len(loc))
print(len(loc2))
print(len(tag))
print(len(timeout))
print(len(career))
print(len(detail))


# print(name)
#print(title)
# print(loc)
# print(loc2)
# tag
# print(timeout)
# print(career)
detail

2
2
2
2
2
2
2
2


['포지션 상세\n[이지스엔터프라이즈를 소개합니다]\n• 이지스엔터프라이즈는 공동주택/집합건물관리 전산솔루션공급 및 금감원에 등록된 전자금융 전문업체로서 전국 약 90% 이상의 시장 점유율을 갖고 있는 독보적인 1위 기업입니다.\n• 공동주택 분야에서 특화된 경쟁력을 기반으로 국내 시장을 석권하고 있는 우량 강소기업입니다.\n• 국내 시장 점유율에 따른 우월적 지위와 새로운 혁신 기술을 이용하여 다양한 서비스를 확장하고자 합니다.\n\n[DX팀은 이렇게 구성되어 있습니다.]\n차세대 ERP 서비스 사업을 성공적으로 추진하기 위하여 TF 조직으로 설립되었습니다.\n• DX팀 목표 : 차세대 ERP 서비스 사업 기획 및 구축\n• DX팀 구성 : 기획팀, 디자인팀, 고객지원팀, API 시스템 개발팀, 어플리케이션 개발팀\n\n[DX팀은 이런 서비스들을 준비하고 있습니다.]\n• XP업무 home(가칭) 신규 구축\n• 상거래 플랫폼(가칭) 신규 구축\n• XP 문서함 고도화 (서비스 간 연계 강화, 상품 완성도 제고)\n• 전자 결재 고도화 (서비스 간 연계 강화, 상품 완성도 제고)\n\n[DX팀의 개발자들은 이렇게 일합니다.]\n• 내부망 / 외부망 분리 운영\n• jira-Confluence 소통\n• SVN, Git 활용\n• Spring 5.0 사용\n• IBSHEET 8.0 그리드 툴 사용\n• UBIFORM 리포트 툴 사용\n주요업무\n• XP업무 home(가칭)을 신규 구축합니다.\n• 상거래 플랫폼(가칭)을 신규 구축합니다.\n• XP 문서함 고도화를 진행합니다. (서비스 간 연계 강화, 상품 완성도 제고)\n• 전자 결재 고도화를 진행합니다. (서비스 간 연계 강화, 상품 완성도 제고)\n자격요건\n• vue.js,vuex 사용 및 scss 업무 경험이 있으신 분 (최소 1년 이상)\n• Spring boot 프레임워크 사용 경험이 있으신 분\n• Git 사용 및 jenkins 활용 경험이 있으신 분\n• jira 및 confluence 활용 경

In [ ]:
1
2
3
4
이
십삼
십사
십오
구
100
200
300
400
800
1000


NameError: name '이' is not defined

In [1]:
100
200
300
400
800
1
2
3
4
900
칠
팔
십육
십칠
십팔
구
100
200
300
400
800
1000
100
200
300
400
800
1000
100
200
300
400
800
1000
100
200
300
400
800
1000
100
200
300
400
800
1000
100
200
300
400
800
1000
100
200
300
400
800
1000
100
200
300
400
800
1000
100
200
300
400
800
1000
100
200
300
400
800
1000
100

NameError: name '칠' is not defined

In [26]:
df = pd.read_csv(f'C:\\Users\\EZEN\\pro\\pro_data\\pro_{url_name_list[7]}.csv')
df

,Unnamed: 0,기업명,채용공고,위치,위치상세,태그,마감일,경력
0,0,해긴,데이터 기획 및 분석 (BI),서울 구로구,구로구 경인로 662 디큐브시티 타워동 39층,"편의시설,자유로운휴가,대출지원,스톡옵션,식대지원,커피·스낵바,건강검진지원,보너스,설...",상시채용,경력 5-10년
1,1,앤서스랩코리아,[ATI]AI Engineer,서울 강남구,"서울특별시 강남구 언주로 609, PAX TOWER 4층","복지포인트,커피·스낵바,식대지원,사내동호회,자기계발지원,유망산업,301~1,000명...",상시채용,경력 5-10년
2,2,페이히어,[200억↑투자] 데이터 엔지니어,서울 강남구,"서울특별시 강남구 강남대로 374, 16층","커피·스낵바,장비지원,산업기능요원,설립4~9년",상시채용,경력 3-10년
3,3,잼팟,BI/DW 담당자,서울 영등포구,서울특별시 영등포구 문래로28길,"식대지원,복지포인트,커피·스낵바,유연근무,설립10년이상",상시채용,경력 5-10년
4,4,앤서스랩코리아,[ATI]MLOps Engineer,서울 강남구,"서울특별시 강남구 언주로 609, PAX TOWER 4층","복지포인트,커피·스낵바,식대지원,사내동호회,자기계발지원,유망산업,301~1,000명...",상시채용,경력 5-10년
5,5,인포유앤컴퍼니,Data Engineer (BI 개발자),서울 성동구,"서울시 성동구 성수일로 111, 선명스퀘어 311호","보너스,사내동호회,유연근무,설립4~9년",상시채용,경력 2-10년
6,6,토스뱅크,Risk System Developer,서울 강남구,서울특별시 강남구 테헤란로 131,"통신비,단체보험,보너스,커피·스낵바,재택근무,편의시설,대출지원,유급휴가,301~1,...",상시채용,경력 2-20년
7,7,인센드,백엔드 개발자,경기 성남시,경기도 성남시 분당구 황새울로319번길 6 410호,NaN,상시채용,경력 8-15년
8,8,인포유앤컴퍼니,PM(Project Manager),서울 성동구,"서울시 성동구 성수일로 111, 선명스퀘어 311호","보너스,사내동호회,유연근무,설립4~9년",상시채용,경력 2-10년
9,9,세이브더칠드런코리아,IT팀 DW/BI 담당자,서울 마포구,서울특별시 마포구 토정로 174,"유연근무,설립10년이상",상시채용,경력 4-15년


In [28]:
df = pd.DataFrame()

In [29]:
df['테스트'] = ['','',1,'일']
df['테스트2'] = ['','',2,'이']

In [32]:
df.to_csv('C:\\Users\\EZEN\\pro\\pro_data\\test1.csv',encoding='utf-8')

In [33]:
df = pd.read_csv('C:\\Users\\EZEN\\pro\\pro_data\\test1.csv',encoding='utf-8')

In [34]:
df

,Unnamed: 0,테스트,테스트2
0,0,NaN,NaN
1,1,NaN,NaN
2,2,1,2
3,3,일,이


In [36]:
driver = webdriver.Chrome("")
time.sleep(3)
driver.get('https://www.wanted.co.kr/wd/219589')
time.sleep(3)

driver.find_element(By.CLASS_NAME, value='Button_Button__interaction__kkYaa').click()

In [48]:
elem = driver.find_element(By.CLASS_NAME, value='JobDescription_JobDescription__VWfcb')

In [50]:
elem.text

"포지션 상세\nCDRI는 화장품 인증과 관련된 솔루션을 자체 개발하고 있습니다. 좋은 제품을 빨리 알릴 수 없을까 고민하다 수출이 가장 빠른 길 중에 하나라는 것을 알 수 있었습니다. 수출을 하려고 인증을 하다 보니 국가마다 법도 다르고 에이전시마다 요구하는 서류도 달랐습니다. 이 모든 절차가 비효율적이라는 것을 깨닫게 되었고, 질문을 하게 됩니다.\n‘왜 인증을 이렇게 밖에 할 수 없을까?’ 하고요.\n분명 저희와 같은 고민을 하고 있는 화장품 제조사 또는 브랜드사들이 있을 것이라고 생각합니다. 그래서 저희는 'certicos'라는 인증 프로그램을 개발하게 되었습니다.\n이제 우리는 인증에 대해 다른 시각으로 바라보아야 합니다.\n자동화 인증, 이것이 CDRI의 핵심입니다.\n\n[CDRI 개발자 이야기]\nCDRI 라이프 블로그 https://blog.naver.com/cdricorp\n\n[CDRI 주요 개발 문화]\n• 애자일 프로세스 기반 프로젝트 진행\n• 온/오프라인 칸반 진행\n• 테크 이슈를 상호 공유하는 주간 테크톡 진행\n• 스킬 업을 위한 그룹 스터디 시간 제공\n• 주기적인 코드 리뷰 진행\n주요업무\n• 화장품 분야 비즈니스의 요구사항에 맞춘 데이터 분석 및 모델 설계\n• 화장품 인증관련 NLP, Computer Vision 업무 알고리즘 연구 개발\n• 화장품 인증관련 글로벌 빅데이터 수집/가공/처리 수행\n• 데이터 품질과 모델 성능 지속적 개선 및  프로덕션 환경에 배포\n자격요건\n• 컴퓨터 공학, 통계학 등 머신 러닝 관련 분야를 전공하신 분\n• 딥러닝 프레임워크 활용에 능숙하신 분(Pytorch, Tensorflow 등)\n• 새로운 기술에 적극적이고 능동적인 분\n• 협업에 적극적이고, 커뮤니케이션을 유연하게 잘 하시는 분\n우대사항\n• 인공지능, 컴퓨터 공학 분야 석사 이상 학위 보유\n• 머신러닝/딥러닝 분야 3년 이상 현업 경력\n• 인공지능 분야 국내외 논문 출판 경험\n혜택 및 복지\n[Benefits]\n

In [68]:
a =' 안녕 하세요'

In [70]:
'안녕하세요' in a

False

In [84]:
df = pd.read_csv(f'C:\\Users\\EZEN\pro\\pro_data\\pro_wanted_web_developer.csv',encoding='utf-8')
df = df.fillna('')
conn = pymysql.connect(host='127.0.0.1', user='user', password='user1234', db='dbmaria', charset='utf8')
cur = conn.cursor()
cur.execute(f'''CREATE TABLE IF NOT EXISTS pro_wanted_web_developer2 (
               name VARCHAR(255),
               title VARCHAR(255),
               time VARCHAR(255),
               loc VARCHAR(255),
               career VARCHAR(255),
               info VARCHAR(255),
               loc2 VARCHAR(255),
               detail TEXT(20000)
            )''')

for index, row in df.iterrows():
    cur.execute(f"""
        INSERT INTO pro_wanted_web_developer2 (
           name,
           title,
           time,
           loc,
           career,
           info,
           loc2,
           detail
        ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s)
    """, (
        row['기업명'],
        row['채용공고'],
        row['마감일'],
        row['위치'],
        row['경력'],
        row['태그'],
        row['위치상세'],
        row['채용상세']
    ))
    
# 변경사항 커밋
conn.commit()

# 연결 종료
conn.close()

In [80]:
df

,Unnamed: 0,기업명,채용공고,위치,위치상세,태그,마감일,경력,채용상세
0,0,이지스엔터프라이즈,[DX팀] 백엔드 개발자(10년 이상),서울 금천구,서울 금천구 가산디지털2로98,"의료비지원,건강검진지원,사내동호회,커피·스낵바,자기계발지원,식대지원,설립10년이상",상시채용,경력 10-15년,포지션 상세\n[이지스엔터프라이즈를 소개합니다]\n• 이지스엔터프라이즈는 공동주택/...
1,1,멋쟁이사자처럼,[TECHIT] 웹 퍼블리셔,서울 종로구,"서울특별시 종로구 종로3길 17, D1동 16층(청진동, D타워)","유연근무,식대지원,커피·스낵바,건강검진지원,유망산업,설립4~9년,자기계발지원",상시채용,경력 3-6년,포지션 상세\n“Possibility to Reality”\n\n저희 멋쟁이사자처럼...
2,2,넥스트플레이어,주니어 웹 개발자(Java Spring),서울 용산구,"서울특별시 용산구 서빙고로 17, 센트럴파크타워 22층","자율복장,커피·스낵바,식대지원,유연근무,인원성장,설립10년이상",상시채용,경력 3-7년,"포지션 상세\n넥스트 플레이어는 든든한 파트너를 통해 좋은 제품을 개발하고, 생산하..."
3,3,섹타나인,"[프론트엔드] 해피포인트 선물하기, 해피마켓 및 해피콘",서울 강남구,"서울시 강남구 논현로 201, SPC2023",NaN,상시채용,경력 4-10년,포지션 상세\n'Make your life smart'\n섹타나인은 더 나은 일상을...
4,4,크리스틴컴퍼니,파이썬 백엔드 개발자 / 2년이상(서울),서울 강남구,강남구 테헤란로306 카이트타워 15층,"커피·스낵바,스톡옵션,보너스,유급휴가,장비지원,리프레시휴가,식대지원,유망산업,설립4~9년",상시채용,경력 2-5년,"포지션 상세\nAnytime, Everywhere, Whatever, You Wan..."


In [11]:
df = pd.read_csv(f'C:\\Users\\EZEN\pro\\pro_data\\pro_{file_name}.csv',encoding='utf-8')

In [12]:
df

,Unnamed: 0,기업명,채용공고,위치,위치상세,태그,마감일,경력,채용상세
0,0,두나무(업비트/증권플러스),MLOps Engineer,서울 강남구,서울시 강남구 테헤란로4길 14 미림타워,"유망산업,건강검진지원,어린이돌봄,대출지원,통신비,식대지원,301~1,000명,설립1...",상시채용,경력 3-20년,포지션 상세\n금융과 기술이 만나는 그 곳에 두나무가 있습니다.\n\n두나무는 가장...
1,1,엘박스,[200억↑투자] ML Engineer (LLM Application),서울 강남구,"서울특별시 강남구 테헤란로 306, 카이트타워 5층","자기계발지원,식대지원,건강검진지원,자유로운휴가,휴가비,유망산업,커피·스낵바,유연근무...",2024.05.23,경력 3-10년,"포지션 상세\n엘박스는 한국의 대표 법률 인공지능 소프트웨어 스타트업으로, 차별화된..."
2,2,한국딥러닝주식회사,AI 엔지니어 (1년 이상),서울 서초구,서초구 강남대로89길 30,"편의시설,보너스,연봉상위2~5%,50명이하,설립4~9년,유망산업",상시채용,경력 1-10년,포지션 상세\n한국딥러닝(KDL) AI솔루션개발팀은 이렇게 일합니다.\n\n• 다양...
3,3,셀렉트스타,AI NLP 연구원,서울 강남구,"서울특별시 강남구 테헤란로20길 20, 삼정빌딩 10F/11F","유망산업,장비지원,유급휴가,식대지원,재택근무,무제한연차,설립4~9년",상시채용,경력 3-5년,"포지션 상세\n셀렉트스타는 ""AI를 통해 IT 산업을 발전시켜 세상을 더 편하게 만..."
4,4,아이스크림에듀,머신러닝 엔지니어,서울 강남구,서울시 강남구 테헤란로 88길 15 (대치동 944-23),"유연근무,워크샵,복지포인트,보너스,설립10년이상",상시채용,경력 2-5년,"포지션 상세\n대한민국 에듀테크 업계를 선도하고 있는 기업, 아이스크림에듀와 함께 ..."
...,...,...,...,...,...,...,...,...,...
209,209,데이블(Dable),"Machine Learning Engineer(전문연 현역/보충역, 산기능 보충역)",서울 강남구,"서울시 강남구 테헤란로 507, 위워크타워 4층","편의시설,유연근무,식대지원,재택근무,건강검진지원,유망산업,복지포인트,설립4~9년",상시채용,신입,포지션 상세\n“글로벌 콘텐츠 디스커버리 플랫폼 데이블에 지원하세요~!”\n\n데이...
210,210,딥브레인에이아이,[400억↑투자] 딥러닝 연구원 / 엔지니어 (전문연구요원 가능),서울 강남구,서울특별시 강남구 역삼로 7길 6,"의료비지원,자기계발지원,유망산업,커피·스낵바,전문연구요원,산업기능요원,설립4~9년",상시채용,신입,"포지션 상세\n딥브레인AI는 대화형 인공 지능 스타트업입니다. 딥러닝 영상 합성, ..."
211,211,딥브레인에이아이,[400억↑투자] 산업기능요원,서울 강남구,서울특별시 강남구 역삼로 7길 6,"의료비지원,자기계발지원,유망산업,커피·스낵바,전문연구요원,산업기능요원,설립4~9년",상시채용,신입,"포지션 상세\n딥브레인AI는 대화형 인공 지능 스타트업입니다. 딥러닝 영상 합성, ..."
212,212,클럼엘,신입 Software Engineer (Data),서울 강남구,"서울 강남구 테헤란로26길 12, 제일비전타워 2층","스톡옵션,자율복장,자유로운휴가,재택근무,커피·스낵바,식대지원,설립3년이하",상시채용,신입,"포지션 상세\nAI 머신러닝 솔루션, 데이터 소프트웨어 엔지니어를 모십니다.\n\n..."


In [15]:
df = pd.read_csv(f'C:\\Users\\EZEN\pro\\pro_data\\pro_{file_name}.csv',encoding='utf-8')
df = df.fillna('')
conn = pymysql.connect(host='127.0.0.1', user='user', password='user1234', db='dbmaria', charset='utf8')
cur = conn.cursor()
cur.execute(f'''CREATE TABLE IF NOT EXISTS pro_{file_name}3 (
               name VARCHAR(255),
               title VARCHAR(255),
               time VARCHAR(255),
               loc VARCHAR(255),
               career VARCHAR(255),
               info VARCHAR(255),
               loc2 VARCHAR(255)
            )''')

for index, row in df.iterrows():
    cur.execute(f"""
        INSERT INTO pro_{file_name}3 (
           name,
           title,
           time,
           loc,
           career,
           info,
           loc2
        ) VALUES (%s, %s, %s, %s, %s, %s, %s)
    """, (
        row['기업명'],
        row['채용공고'],
        row['마감일'],
        row['위치'],
        row['경력'],
        row['태그'],
        row['위치상세']
    ))
# 변경사항 커밋
conn.commit()

# 연결 종료
conn.close()

In [38]:
conn = pymysql.connect(host='127.0.0.1', user='user', password='user1234', db='dbmaria', charset='utf8')
cur = conn.cursor()
cur.execute('''SELECT * FROM pro_wanted_ml_engineer3
            ''')
results = cur.fetchall()


(('두나무(업비트/증권플러스)', 'MLOps Engineer', '상시채용', '서울 강남구', '경력 3-20년', '유망산업,건강검진지원,어린이돌봄,대출지원,통신비,식대지원,301~1,000명,설립10년이상', '서울시 강남구 테헤란로4길 14 미림타워'), ('엘박스', '[200억↑투자] ML Engineer (LLM Application)', '2024.05.23', '서울 강남구', '경력 3-10년', '자기계발지원,식대지원,건강검진지원,자유로운휴가,휴가비,유망산업,커피·스낵바,유연근무,설립4~9년', '서울특별시 강남구 테헤란로 306, 카이트타워 5층'), ('한국딥러닝주식회사', 'AI 엔지니어 (1년 이상)', '상시채용', '서울 서초구', '경력 1-10년', '편의시설,보너스,연봉상위2~5%,50명이하,설립4~9년,유망산업', '서초구 강남대로89길 30'), ('셀렉트스타', 'AI NLP 연구원', '상시채용', '서울 강남구', '경력 3-5년', '유망산업,장비지원,유급휴가,식대지원,재택근무,무제한연차,설립4~9년', '서울특별시 강남구 테헤란로20길 20, 삼정빌딩 10F/11F'), ('아이스크림에듀', '머신러닝 엔지니어', '상시채용', '서울 강남구', '경력 2-5년', '유연근무,워크샵,복지포인트,보너스,설립10년이상', '서울시 강남구 테헤란로 88길 15 (대치동 944-23)'), ('파인더스에이아이', 'Computer Vision AI Engineer_Junior', '2024.05.31', '서울 서초구', '경력 5-12년', '스톡옵션,식대지원,건강검진지원,커피·스낵바,설립4~9년', '서초구 강남대로 51길 1, 13층(서초동, 511타워)'), ('덴컴', 'Front-end 개발자', '상시채용', '서울 강남구', '경력 4-10년', '유연근무,리프레시휴가,편의시설,식대지원,유망산업,스톡옵션,커피·스낵바,설립4~9년', '서울특별시 강남구 삼성로85길 26, V&S 14층'), ('버즈빌(

In [18]:
conn = pymysql.connect(host='127.0.0.1', user='user', password='user1234', db='dbmaria', charset='utf8')
cur = conn.cursor()
cur.execute('''SELECT * FROM pro_wanted_ml_engineer3
            ''')

results = cur.fetchall()

def search_loc():
    question = input('지역을 입력해주세요\n')
    answer = []

    for i in range(len(results)):
        if question in(str(results[i][3])):
            answer.append(str(results[i][0]))
    return answer

In [19]:
search_loc()

지역을 입력해주세요
구로구


['더매트릭스', '미리디', '딥바이오', '한국해양기상기술']

In [14]:
question = input('지역을 입력해주세요\n')
answer = []

for i in range(len(results)):
    if question in(str(results[i][3])):
        answer.append(str(results[i]))
answer

지역을 입력해주세요
강남구


["('두나무(업비트/증권플러스)', 'MLOps Engineer', '상시채용', '서울 강남구', '경력 3-20년', '유망산업,건강검진지원,어린이돌봄,대출지원,통신비,식대지원,301~1,000명,설립10년이상', '서울시 강남구 테헤란로4길 14 미림타워')",
 "('엘박스', '[200억↑투자] ML Engineer (LLM Application)', '2024.05.23', '서울 강남구', '경력 3-10년', '자기계발지원,식대지원,건강검진지원,자유로운휴가,휴가비,유망산업,커피·스낵바,유연근무,설립4~9년', '서울특별시 강남구 테헤란로 306, 카이트타워 5층')",
 "('셀렉트스타', 'AI NLP 연구원', '상시채용', '서울 강남구', '경력 3-5년', '유망산업,장비지원,유급휴가,식대지원,재택근무,무제한연차,설립4~9년', '서울특별시 강남구 테헤란로20길 20, 삼정빌딩 10F/11F')",
 "('아이스크림에듀', '머신러닝 엔지니어', '상시채용', '서울 강남구', '경력 2-5년', '유연근무,워크샵,복지포인트,보너스,설립10년이상', '서울시 강남구 테헤란로 88길 15 (대치동 944-23)')",
 "('덴컴', 'Front-end 개발자', '상시채용', '서울 강남구', '경력 4-10년', '유연근무,리프레시휴가,편의시설,식대지원,유망산업,스톡옵션,커피·스낵바,설립4~9년', '서울특별시 강남구 삼성로85길 26, V&S 14층')",
 "('브레인커머스', '[잡플래닛] AI Engineer', '상시채용', '서울 강남구', '경력 3-10년', '식대지원,의료비지원,건강검진지원,커피·스낵바,스톡옵션,설립10년이상', '서울 강남구 테헤란로 509 10층')",
 "('웨이센(waycenInc.)', 'Medical AI Research Engineer(Senior)', '상시채용', '서울 강남구', '경력 5-15년', '커피·스낵바,건강검진지원,식대지원,편의시설,유망산업,설립4~9년', '서울시 강남구

In [7]:
driver = webdriver.Chrome("")

In [8]:
driver.get('https://www.wanted.co.kr/wd/220357')

In [9]:
elem = driver.find_element(By.CLASS_NAME, value='JobSkillTags_JobSkillTags__V1tZr')            

In [10]:
elem.text

'기술 스택 • 툴\nConfluence\nGit\nJIRA\nLinux\nMySQL\nOracle\nRedis\nJava\nJavaScript\nJenkins\nDocker\nSpring Framework\nRabbitMQ\nERP 소프트웨어\nVue.JS'